In [ ]:
import os
import librosa
import numpy as np
from scipy.signal import find_peaks
import random

features = ["BPM_mean", "BPM_std", "ZCR_mean", "ZCR_std", "Spectral_Centroid_mean", "Spectral_Bandwidth_mean", "Spectral_Rolloff_mean", "Spectral_Contrast_mean", "Spectral_Flatness_mean", "RMS_mean", "Chroma_STFT_mean", "MFCC_mean"]

def calculate_bpm(y, sr):
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)
    return tempo

def calculate_audio_features(file_path, chunk_size=30, hop_length=512, n_fft=2048):
    y, sr = librosa.load(file_path, sr=None, mono=True)
    total_duration = librosa.get_duration(y=y, sr=sr)

    # Randomly select a 30-second chunk from the audio file
    if total_duration > chunk_size:
        start = random.uniform(0, total_duration - chunk_size)
    else:
        start = 0
    
    end = start + chunk_size
    start_sample = librosa.time_to_samples(start, sr=sr)
    end_sample = librosa.time_to_samples(end, sr=sr)
    y = y[start_sample:end_sample]

    # Calculate the BPM for the entire track
    # BPM = librosa.feature.tempo(y=y, sr=sr, hop_length=hop_length, aggregate=None)
    BPM = calculate_bpm(y, sr)

    # Calculate the ZCR for the entire track
    zcr = librosa.feature.zero_crossing_rate(y=y) 

    # Calculate the spectral centroid for the entire track
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)

    # Calculate the spectral bandwidth for the entire track
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)

    # Calculate the rms energy for the entire track
    rms = librosa.feature.rms(y=y, hop_length=hop_length)

    # Calculate the spectral roll-off for the entire track
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)

    # Calculate the chroma stft for the entire track
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)

    #Calculate the spectral contrast for the entire track
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)

    # Calculate the spectral flatness for the entire track
    spectral_flatness = librosa.feature.spectral_flatness(y=y, n_fft=n_fft, hop_length=hop_length)

    # Calculate the mel-frequency cepstral coefficients for the entire track
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)

    return (np.mean(BPM), np.std(BPM), np.mean(zcr), np.std(zcr), np.mean(spectral_centroid), np.mean(spectral_bandwidth), np.mean(spectral_rolloff), np.mean(spectral_contrast), np.mean(spectral_flatness), np.mean(rms), np.mean(chroma_stft), np.mean(mfcc))

def process_directories(directories):
    for directory_path in directories:
        output_file_path = os.path.join(directory_path, "results.txt")
        with open(output_file_path, 'w') as output_file:
            output_file.write('\t'.join(features) + '\n')
            for filename in os.listdir(directory_path): # For each file in the directory
                if filename.lower().endswith('.wav'):
                    file_path = os.path.join(directory_path, filename)
                    print(f"Processing {file_path}")
                    results = calculate_audio_features(file_path)
                    output_file.write('\t'.join([str(result) for result in results]) + '\n')
        print(f"Results saved to {output_file_path}")

def main():
    with open('/Volumes/SanDisk/PunkMusic/directory.txt', 'r') as file:
        directories = [line.strip() for line in file]
    process_directories(directories)

if __name__ == "__main__":
    main()
